###  **Table of Contents**

- **Basic autograd example 1**

- **Basic autograd example 2**

- **Loading data from numpy**

- **Input pipline**

- **Input pipline for custom dataset**

- **Pretrained model**

- **Save and load model**

In [71]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

---------------------------------------------

### **1. Basic autograd example 1**

In [70]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [72]:
# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

In [73]:
# Compute gradients.
y.backward()

In [74]:
# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(1.)
tensor(1.)


-------------------------------------

### **2. Basic autograd example 2**

In [75]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

In [77]:
# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

w:  Parameter containing:
tensor([[ 0.2876,  0.1621, -0.4114],
        [-0.5443,  0.3746, -0.1072]], requires_grad=True)
b:  Parameter containing:
tensor([-0.3802, -0.3896], requires_grad=True)


In [79]:
# Inspecting Model Architecture
linear

Linear(in_features=3, out_features=2, bias=True)

In [80]:
# Build loss function and optimizer.
criterion = nn.MSELoss()    # Mean Squared Loss
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)   # Stochastic Gradient Descent 

In [81]:
# Forward pass.
pred = linear(x)
pred

tensor([[-0.4796, -1.8690],
        [ 0.1391, -0.0610],
        [-0.3085, -1.0829],
        [-0.3255, -0.6468],
        [-0.8768, -0.2775],
        [-1.1618, -0.6427],
        [-0.4697, -2.0710],
        [-0.2679,  0.2660],
        [-0.5521, -0.2107],
        [-0.5487,  0.5777]], grad_fn=<AddmmBackward0>)

In [82]:
# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

loss:  2.3966920375823975


In [83]:
# Backward pass.
loss.backward()

In [84]:
# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

dL/dw:  tensor([[ 0.0772, -0.0893, -0.5444],
        [-0.8112,  0.9380, -0.1872]])
dL/db:  tensor([-1.2305, -0.7609])


In [85]:
# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  2.3572089672088623


----------------------------------------

### **3. Loading data from numpy**

In [88]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])
x

array([[1, 2],
       [3, 4]])

In [89]:
# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)
y

tensor([[1, 2],
        [3, 4]])

In [90]:
# Convert the torch tensor to a numpy array.
z = y.numpy()
z

array([[1, 2],
       [3, 4]])

------------------------------------

### **4. Input pipeline**

In [92]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

Files already downloaded and verified


In [125]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../../data/
    Split: Train
    StandardTransform
Transform: ToTensor()

-------------------------------

- `train_dataset[i][j]` represents accessing the `j`-th data component of the `i`-th sample in the CIFAR-10 training dataset.

  - `i`: Index representing the position of the sample in the dataset. Indexing starts from 0, so `i=0` corresponds to the first sample, `i=1` corresponds to the second sample, and so on.

  - `j`: Also an index, indicating which data component to access, whether it's image data or labels. In the CIFAR-10 dataset, `j=0` corresponds to image data, and `j=1` corresponds to label data.

Examples:

- `train_dataset[0][0]`: Accesses the image data of the first sample.
- `train_dataset[0][1]`: Accesses the label data of the first sample.
- `train_dataset[1][0]`: Accesses the image data of the second sample.
- `train_dataset[1][1]`: Accesses the label data of the second sample.

When accessing samples in the dataset, `train_dataset[i]` returns a tuple containing both image and label data, allowing you to use `[j]` to specify the specific part you want to access.

In [111]:
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([3, 32, 32])
6


In [126]:
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [127]:
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

In [129]:
# Mini-batch images and labels.

# The source code here, `images, labels = data_iter.next()`, has been deprecated in the latest version.
images, labels = next(data_iter)

In [132]:
images.shape,labels.shape

(torch.Size([64, 3, 32, 32]), torch.Size([64]))

In [133]:
# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

---------------------------------

### **5. Input pipeline for custom dataset**

In [134]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

In [ ]:
# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

---------------------------------

### **6. Pretrained model**

In [136]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

In [139]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [142]:
# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

In [143]:
# Replace the top layer for fine-tuning.
# Replace the last fully connected layer of ResNet with a new linear layer
# that has the specified input feature count and output feature count.
# In this example, the input feature count is resnet.fc.in_features, and the
# output feature count is 100.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.


In [144]:
# Forward pass.
images = torch.randn(64, 3, 224, 224)   # batch size, channels, height, width
outputs = resnet(images)
print (outputs.size())     # (64, 100)

torch.Size([64, 100])


------------------------------

### **7. Save and load the model**

In [ ]:
# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

In [ ]:
# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))